#Customer Churn Predictor

In [1]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=eb75f9a87366b89d946c182c896958ae0ae560d78597d568151a32845f130724
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


#Creating SparkSession

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('customer_churn').getOrCreate()

#Reading Data From CSV

In [4]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [5]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [7]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [6]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

#Creating Feature Vector Using VectorAssembler

In [9]:
from pyspark.ml.feature import VectorAssembler

In [10]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [11]:
output = assembler.transform(data)

In [12]:
output.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|            features|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|[42.0,11066.8,0.0...|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|[41.0,11916.22,0....|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|[38.0,12884.75,0....|
|      Phillip White|4

#Filtering out the other columns for training except features and Churn

In [16]:
final_data = output.select(['features','Churn'])
final_data.show()

+--------------------+-----+
|            features|Churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



#Splitting The Dataframe in Train And Test

In [18]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

#Creating a Logistic Regression Model

In [19]:
from pyspark.ml.classification import LogisticRegression

In [20]:
lor_model = LogisticRegression(labelCol='Churn')

#Training the Model using train dataframe

In [21]:
lor_trained_fit = lor_model.fit(train_data)

#Evaluating the model using test dataframe

In [23]:
results = lor_trained_fit.evaluate(test_data)

In [24]:
results.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[25.0,9672.03,0.0...|    0|[3.92025351616161...|[0.98054975106509...|       0.0|
|[28.0,11128.95,1....|    0|[3.64517655369496...|[0.97454792819925...|       0.0|
|[28.0,11245.38,0....|    0|[3.11016994440148...|[0.95731030148829...|       0.0|
|[29.0,10203.18,1....|    0|[3.26352654754519...|[0.96315613975357...|       0.0|
|[29.0,11274.46,1....|    0|[4.00527989205094...|[0.98210681026924...|       0.0|
|[30.0,10183.98,1....|    0|[2.39208337328767...|[0.91622162565107...|       0.0|
|[30.0,12788.37,0....|    0|[1.90045156568415...|[0.86994262542212...|       0.0|
|[31.0,5304.6,0.0,...|    0|[2.82411022884699...|[0.94396487491168...|       0.0|
|[31.0,8688.21,0.0...|    0|[6.12745634777249...|[0.99782262682617...|       0.0|
|[31.0,9574.89,0

#Measuring the ROC using BinaryClassificationEvaluator for evaluating the test results

In [25]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [29]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',labelCol='Churn')

In [31]:
ROC = my_eval.evaluate(results.predictions)

In [32]:
ROC

0.9123170998034505

#Deploying the model using Unlabeled Data

In [35]:
unlabeled_data = spark.read.csv('new_customers.csv',inferSchema=True,header=True)

In [36]:
unlabeled_data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company']

In [37]:
unlabeled_output = assembler.transform(unlabeled_data) 

In [38]:
unlabeled_output.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'features']

In [46]:
unlabeled_output.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|[65.0,100.0,1.0,1...|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   

In [47]:
new_testset = lor_trained_fit.transform(unlabeled_output)

In [48]:
new_testset.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [49]:
new_testset.select(['Company','prediction']).show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

